#### **MMDetection 설치**

In [ ]:
!pip install mmcv-full
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

In [1]:
from mmdet.apis import init_detector,inference_detector
import mmcv

#### **Oxford Pet Dataset 다운로드**

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
!mkdir /content/data
!tar -xvf images.tar.gz -C /content/data
!tar -xvf annotations.tar.gz -C /content/data

#### **이미지 디렉토리와 annotation 파일 살펴 보기**

In [4]:
!ls -lia ./data/images/Abyss*.jpg

3804141 -rwxr-xr-x 1 1001 1001 126923 Jun 18  2012 ./data/images/Abyssinian_100.jpg
3823403 -rwxr-xr-x 1 1001 1001  49023 Jun 18  2012 ./data/images/Abyssinian_101.jpg
3824479 -rwxr-xr-x 1 1001 1001 165505 Jun 18  2012 ./data/images/Abyssinian_102.jpg
3825251 -rwxr-xr-x 1 1001 1001 132937 Jun 18  2012 ./data/images/Abyssinian_103.jpg
3825401 -rwxr-xr-x 1 1001 1001  34907 Jun 18  2012 ./data/images/Abyssinian_104.jpg
3823782 -rwxr-xr-x 1 1001 1001 141823 Jun 18  2012 ./data/images/Abyssinian_105.jpg
3803224 -rwxr-xr-x 1 1001 1001 197092 Jun 18  2012 ./data/images/Abyssinian_106.jpg
3825292 -rwxr-xr-x 1 1001 1001 150955 Jun 18  2012 ./data/images/Abyssinian_107.jpg
3805516 -rwxr-xr-x 1 1001 1001 120641 Jun 18  2012 ./data/images/Abyssinian_108.jpg
3823771 -rwxr-xr-x 1 1001 1001  13601 Jun 18  2012 ./data/images/Abyssinian_109.jpg
3802023 -rwxr-xr-x 1 1001 1001  96294 Jun 18  2012 ./data/images/Abyssinian_10.jpg
3805469 -rwxr-xr-x 1 1001 1001  27734 Jun 18  2012 ./data/images/Abyssinian_1

In [5]:
!ls -lia ./data/images

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3803968 -rwxr-xr-x 1 1001 1001   27987 Jun 18  2012 Egyptian_Mau_91.jpg
3826712 -rwxr-xr-x 1 1001 1001   31662 Jun 18  2012 Egyptian_Mau_92.jpg
3825416 -rwxr-xr-x 1 1001 1001   11833 Jun 18  2012 Egyptian_Mau_93.jpg
3804946 -rwxr-xr-x 1 1001 1001   13282 Jun 18  2012 Egyptian_Mau_94.jpg
3824403 -rwxr-xr-x 1 1001 1001   25425 Jun 18  2012 Egyptian_Mau_95.jpg
3803133 -rwxr-xr-x 1 1001 1001   22833 Jun 18  2012 Egyptian_Mau_96.jpg
3803462 -rwxr-xr-x 1 1001 1001  181499 Jun 18  2012 Egyptian_Mau_97.jpg
3805737 -rwxr-xr-x 1 1001 1001    8709 Jun 18  2012 Egyptian_Mau_98.jpg
3805943 -rwxr-xr-x 1 1001 1001   12394 Jun 18  2012 Egyptian_Mau_99.jpg
3824115 -rwxr-xr-x 1 1001 1001   48984 Jun 18  2012 Egyptian_Mau_9.jpg
3824256 -rwxr-xr-x 1 1001 1001  164254 Jun 18  2012 english_cocker_spaniel_100.jpg
3805852 -rwxr-xr-x 1 1001 1001  170083 Jun 18  2012 english_cocker_spaniel_101.jpg
3823164 -rwxr-xr-x 1 1001 1001  241519 Jun 18  2012 english_cocker_spaniel_102.

In [9]:
!cat ./data/annotations/xmls/Abyssinian_1.xml

<annotation><folder>OXIIIT</folder><filename>Abyssinian_1.jpg</filename><source><database>OXFORD-IIIT Pet Dataset</database><annotation>OXIIIT</annotation><image>flickr</image></source><size><width>600</width><height>400</height><depth>3</depth></size><segmented>0</segmented><object><name>cat</name><pose>Frontal</pose><truncated>0</truncated><occluded>0</occluded><bndbox><xmin>333</xmin><ymin>72</ymin><xmax>425</xmax><ymax>158</ymax></bndbox><difficult>0</difficult></object></annotation>


In [13]:
import glob
import xml.etree.ElementTree as ET

def get_bboxes_from_xml_test(xml_file):
  tree=ET.parse(xml_file)
  root=tree.getroot()
  bbox_names=[]
  bboxes=[]
  for obj in root.findall('object'):
    bbox_name=obj.find('name').text
    xmlbox=obj.find('bndbox')
    x1=int(xmlbox.find('xmin').text)
    y1=int(xmlbox.find('ymin').text)
    x2=int(xmlbox.find('xmax').text)
    y2=int(xmlbox.find('ymax').text)

    bbox_names.append(bbox_name)
    bboxes.append([x1,y1,x2,y2])

  return bbox_names,bboxes

get_bboxes_from_xml_test('./data/annotations/xmls/Abyssinian_1.xml')

(['cat'], [[333, 72, 425, 158]])

#### **trian,val image/annotation 메타 파일 보기**
* train과 valid 데이터로 나뉠 image와 annotation의 파일명을 가지는 메타 파일
* train과 valid용 meta 파일을 별도로 만듬

In [14]:
!cd ./data/annotations; cat trainval.txt

Abyssinian_100 1 1 1
Abyssinian_101 1 1 1
Abyssinian_102 1 1 1
Abyssinian_103 1 1 1
Abyssinian_104 1 1 1
Abyssinian_105 1 1 1
Abyssinian_106 1 1 1
Abyssinian_107 1 1 1
Abyssinian_108 1 1 1
Abyssinian_109 1 1 1
Abyssinian_10 1 1 1
Abyssinian_110 1 1 1
Abyssinian_111 1 1 1
Abyssinian_112 1 1 1
Abyssinian_113 1 1 1
Abyssinian_114 1 1 1
Abyssinian_115 1 1 1
Abyssinian_116 1 1 1
Abyssinian_117 1 1 1
Abyssinian_118 1 1 1
Abyssinian_119 1 1 1
Abyssinian_11 1 1 1
Abyssinian_120 1 1 1
Abyssinian_121 1 1 1
Abyssinian_122 1 1 1
Abyssinian_123 1 1 1
Abyssinian_124 1 1 1
Abyssinian_125 1 1 1
Abyssinian_126 1 1 1
Abyssinian_127 1 1 1
Abyssinian_128 1 1 1
Abyssinian_129 1 1 1
Abyssinian_12 1 1 1
Abyssinian_130 1 1 1
Abyssinian_131 1 1 1
Abyssinian_132 1 1 1
Abyssinian_133 1 1 1
Abyssinian_134 1 1 1
Abyssinian_135 1 1 1
Abyssinian_136 1 1 1
Abyssinian_137 1 1 1
Abyssinian_138 1 1 1
Abyssinian_139 1 1 1
Abyssinian_13 1 1 1
Abyssinian_140 1 1 1
Abyssinian_141 1 1 1
Abyssinian_142 1 1 1
Abyssinian_143 1 

In [20]:
import pandas as pd

pet_df=pd.read_csv('./data/annotations/trainval.txt',sep=' ',header=None,names=['img_name','class_id','etc1','etc2'])
pet_df.head()

,img_name,class_id,etc1,etc2
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1


In [21]:
pet_df['class_id'].value_counts()

37    100
22    100
34    100
32    100
30    100
28    100
26    100
24    100
20    100
35    100
18    100
16    100
14    100
10    100
6     100
4     100
36    100
1     100
3     100
19    100
31    100
29    100
27    100
25    100
5     100
21    100
17    100
15    100
11    100
9     100
7     100
2     100
33     99
23     96
13     96
8      96
12     93
Name: class_id, dtype: int64

In [22]:
pet_df['class_name']=pet_df['img_name'].apply(lambda x: x[:x.rfind('_')])
pet_df.head()

,img_name,class_id,etc1,etc2,class_name
0,Abyssinian_100,1,1,1,Abyssinian
1,Abyssinian_101,1,1,1,Abyssinian
2,Abyssinian_102,1,1,1,Abyssinian
3,Abyssinian_103,1,1,1,Abyssinian
4,Abyssinian_104,1,1,1,Abyssinian


In [25]:
from sklearn.model_selection import train_test_split

train_df,val_df=train_test_split(pet_df,test_size=0.1,stratify=pet_df['class_id'],random_state=2021)

In [26]:
print(train_df['class_id'].value_counts(),val_df['class_id'].value_counts())

37    90
22    90
34    90
32    90
30    90
28    90
26    90
24    90
20    90
35    90
18    90
16    90
14    90
10    90
6     90
4     90
36    90
1     90
3     90
19    90
31    90
29    90
27    90
25    90
5     90
21    90
17    90
15    90
11    90
9     90
7     90
2     90
33    89
23    87
13    86
8     86
12    84
Name: class_id, dtype: int64 37    10
36    10
17    10
16    10
15    10
14    10
13    10
11    10
10    10
9     10
8     10
7     10
6     10
5     10
4     10
3     10
2     10
18    10
19    10
20    10
21    10
35    10
34    10
33    10
32    10
31    10
30    10
29    10
28    10
27    10
26    10
25    10
24    10
22    10
1     10
12     9
23     9
Name: class_id, dtype: int64


In [29]:
train_df=train_df.sort_values(by='img_name')
val_df=val_df.sort_values(by='img_name')

In [30]:
train_df['img_name'].to_csv('./data/train.txt',sep=' ',header=False,index=False)
val_df['img_name'].to_csv('./data/val.txt',sep=' ',header=False,index=False)

In [33]:
pet_classes_list=pet_df['class_name'].unique().tolist()
print(pet_classes_list)

['Abyssinian', 'american_bulldog', 'american_pit_bull_terrier', 'basset_hound', 'beagle', 'Bengal', 'Birman', 'Bombay', 'boxer', 'British_Shorthair', 'chihuahua', 'Egyptian_Mau', 'english_cocker_spaniel', 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese', 'japanese_chin', 'keeshond', 'leonberger', 'Maine_Coon', 'miniature_pinscher', 'newfoundland', 'Persian', 'pomeranian', 'pug', 'Ragdoll', 'Russian_Blue', 'saint_bernard', 'samoyed', 'scottish_terrier', 'shiba_inu', 'Siamese', 'Sphynx', 'staffordshire_bull_terrier', 'wheaten_terrier', 'yorkshire_terrier']


In [34]:
!echo 'train list #####'; cat ./data/train.txt

train list #####
Abyssinian_1
Abyssinian_10
Abyssinian_101
Abyssinian_102
Abyssinian_103
Abyssinian_104
Abyssinian_105
Abyssinian_106
Abyssinian_107
Abyssinian_108
Abyssinian_109
Abyssinian_110
Abyssinian_111
Abyssinian_112
Abyssinian_113
Abyssinian_114
Abyssinian_115
Abyssinian_116
Abyssinian_117
Abyssinian_118
Abyssinian_119
Abyssinian_12
Abyssinian_120
Abyssinian_121
Abyssinian_123
Abyssinian_124
Abyssinian_125
Abyssinian_126
Abyssinian_128
Abyssinian_129
Abyssinian_13
Abyssinian_130
Abyssinian_131
Abyssinian_132
Abyssinian_133
Abyssinian_134
Abyssinian_135
Abyssinian_136
Abyssinian_137
Abyssinian_138
Abyssinian_139
Abyssinian_14
Abyssinian_141
Abyssinian_142
Abyssinian_143
Abyssinian_144
Abyssinian_145
Abyssinian_146
Abyssinian_148
Abyssinian_149
Abyssinian_15
Abyssinian_150
Abyssinian_151
Abyssinian_152
Abyssinian_154
Abyssinian_155
Abyssinian_156
Abyssinian_157
Abyssinian_159
Abyssinian_16
Abyssinian_160
Abyssinian_161
Abyssinian_164
Abyssinian_165
Abyssinian_166
Abyssinian_167
A

In [35]:
!echo 'valid list #####'; cat ./data/val.txt

valid list #####
Abyssinian_100
Abyssinian_11
Abyssinian_122
Abyssinian_127
Abyssinian_140
Abyssinian_153
Abyssinian_158
Abyssinian_182
Abyssinian_190
Abyssinian_193
Bengal_112
Bengal_127
Bengal_128
Bengal_139
Bengal_143
Bengal_145
Bengal_149
Bengal_169
Bengal_180
Bengal_190
Birman_113
Birman_120
Birman_123
Birman_145
Birman_149
Birman_151
Birman_159
Birman_160
Birman_171
Birman_190
Bombay_10
Bombay_110
Bombay_121
Bombay_122
Bombay_129
Bombay_133
Bombay_140
Bombay_143
Bombay_168
Bombay_174
British_Shorthair_103
British_Shorthair_136
British_Shorthair_141
British_Shorthair_15
British_Shorthair_155
British_Shorthair_17
British_Shorthair_18
British_Shorthair_185
British_Shorthair_199
British_Shorthair_209
Egyptian_Mau_100
Egyptian_Mau_102
Egyptian_Mau_111
Egyptian_Mau_118
Egyptian_Mau_127
Egyptian_Mau_154
Egyptian_Mau_161
Egyptian_Mau_192
Egyptian_Mau_193
Maine_Coon_112
Maine_Coon_129
Maine_Coon_132
Maine_Coon_153
Maine_Coon_173
Maine_Coon_193
Maine_Coon_196
Maine_Coon_197
Maine_Coon_207


#### **mmdetection의 중립 annotaion 포맷 변환**

In [36]:
import xml.etree.ElementTree as ET

# 1개의 annotation 파일에서 bbox 정보 추출. 여러개의 object가 있을 경우 이들 object의 name과 bbox 좌표들을 list로 반환.
def get_bboxes_from_xml(anno_dir, xml_file):
  anno_xml_file = osp.join(anno_dir, xml_file)
  tree = ET.parse(anno_xml_file)
  root = tree.getroot()
  bbox_names = []
  bboxes = []

  # 파일내에 있는 모든 object Element를 찾음. 
  for obj in root.findall('object'):
    #obj.find('name').text는 cat 이나 dog을 반환     
    #bbox_name = obj.find('name').text
    # object의 클래스명은 파일명에서 추출. 
    bbox_name = xml_file[:xml_file.rfind('_')]

    xmlbox = obj.find('bndbox')
    x1 = int(xmlbox.find('xmin').text)
    y1 = int(xmlbox.find('ymin').text)
    x2 = int(xmlbox.find('xmax').text)
    y2 = int(xmlbox.find('ymax').text)

    bboxes.append([x1, y1, x2, y2])
    bbox_names.append(bbox_name)

  return bbox_names, bboxes

In [ ]:
import copy
import os.path as osp
import mmcv
import numpy as np
import cv2

from mmdet.datasets.builder import dataset
from mmdet.datasets.custom import CustomDataset

import xml.etree.ElementTree as ET

PET_CLASSES=pet_df['class_name'].unique().tolist()

@DATASETS.register_module(force=True)
class PetDataset(CustomDataset):
  CLASSES=PET_CLASSES

  def load_annotations(self,ann_file):
    cat2label={K:i for i,k in enumerate(self.CALSSES)}
    image_list=mmcv.list_from_file(self.ann_file))
    data_infos=[]

    for image_id in image_list:
      filename='{0:}/{1:}.jpg'.format(self.img_prefix,image_id)
      image=cv2.imread('filename')
      height,width=image.shape[:2]

      data_info={'filename':filename.
                 'width':width,
                 'height':height}
      label_prefix=self.img_prefix.replace('images','annotatoins')

      anno_xml_file=osp.join(label_prefix,'xmls/'+str(image_id)+'.xml')

      if not osp.exists(anno_xml_file):
        